In [2]:
import sys, os
from word2vec.main import run_program

# Word2Vec con Skip-Gram: implementación y análisis

En este trabajo he implementado desde cero un modelo Word2Vec basado en Skip-Gram con el objetivo de entender su funcionamiento interno. Se ha trabajado con un corpus controlado, diseñado para contener patrones lingüísticos claros como relaciones de género gramatical, asociaciones entre agentes y acciones, y relaciones geográficas simples.

Se ha utilizado softmax completo sobre todo el vocabulario, siendo consciente de su coste computacional, porque permite seguir de forma directa la función objetivo original y analizar con claridad el proceso de entrenamiento. Alternativas más eficientes como Negative Sampling no se han incluido para priorizar la comprensión conceptual del modelo.

La evaluación combina un análisis cuantitativo, mediante la pérdida final y la similitud media entre palabras y sus vecinos más cercanos, con un análisis cualitativo basado en vecinos y analogías no triviales, diseñadas en función del contenido del corpus. Los resultados muestran que el modelo captura regularidades distribucionales básicas, aunque también evidencian las limitaciones derivadas del tamaño del corpus.

In [3]:
base_dir = os.path.abspath(os.path.join(os.getcwd()))
sys.path.insert(0, os.path.join(base_dir))
sys.path.insert(0, os.path.join(base_dir, 'w2v'))
print('Base directory:', base_dir)

Base directory: C:\Users\ralme\OneDrive\Escritorio\4_GCID\Asignaturas\PLN\Practica2_PLN


## Estructura del proyecto

1. Configuración del entorno y carga del corpus  
2. Ejecución del pipeline completo de Word2Vec  
3. Evaluación cuantitativa de los embeddings  
4. Análisis cualitativo: vecinos y analogías  

In [2]:
corpus_path = os.path.join(base_dir, 'resources', 'dataset_word2vec.txt')
if not os.path.exists(corpus_path):
    sample = [
        "París es la capital de Francia",
        "Madrid es la capital de España",
        "el perro ladra en la casa",
        "el gato maúlla en la silla",
        "el coche del conductor está en la calle"
    ]
    os.makedirs(os.path.dirname(corpus_path), exist_ok=True)
    with open(corpus_path, 'w', encoding='utf-8') as f:
        f.write("\n".join(sample))
    print("Sample corpus created at:", corpus_path)

model, vocab, inv_vocab, pairs = run_program(
    corpus_path=corpus_path,
    window_size=2,
    embedding_dim=50,
    learning_rate=0.05,
    epochs=100,
    min_count=1
)

**********************************************************************
WORD2VEC - EJECUCIÓN DEL PROGRAMA MEDIANTE SKIP-GRAM
**********************************************************************

 Primer paso) Cargar y tokenizar el corpus

 Segundo paso) Construir el vocabulario
Tamaño del vocabulario: 355
Top 10 palabras: [('la', 131), ('el', 129), ('es', 37), ('está', 27), ('en', 26), ('un', 20), ('por', 15), ('una', 15), ('al', 14), ('gato', 11)]

 Tercer paso) Generar el par 'Centro-Contexto'
Total pairs: 2413

 Cuarto paso) Entrenar el modelo Skip-Gram

 Quinto paso) Evaluación cuantitativa de los embeddings
Average neighbor similarity: 0.9522

**********************************************************************
ANÁLISIS DE LOS EMBEDDINGS
**********************************************************************

 ********** VECINOS MÁS CERCANOS **********

'perro':
  - gato: 0.9829
  - en: 0.9591
  - tren: 0.9478
  - coche: 0.9471
  - está: 0.9449

'gato':
  - perro: 0.9829
  - en:

# Conclusiones

Los resultados obtenidos indican que el modelo es capaz de aprender relaciones básicas entre las palabras a partir de su uso en el corpus. En las palabras más frecuentes, los vecinos más cercanos suelen tener un significado relacionado, y las analogías planteadas permiten observar relaciones sencillas como el género gramatical, acciones asociadas a personas o animales y algunas relaciones geográficas simples.

El análisis realizado muestra también que el entrenamiento funciona de forma adecuada y que la elección de ciertos parámetros, como el tamaño de la ventana o la dimensión del embedding, influye en la calidad de los resultados. En general, configuraciones con embeddings algo más grandes tienden a ofrecer representaciones más coherentes, aunque esto implica un mayor coste de cálculo.

Al mismo tiempo, el trabajo pone de manifiesto algunas limitaciones. El uso de softmax completo hace que el entrenamiento sea más costoso y poco escalable a corpus grandes, y el tamaño limitado del corpus utilizado condiciona la fiabilidad de analogías más complejas. Estas limitaciones son esperables dado el enfoque del trabajo y el contexto académico en el que se ha desarrollado.

Como posible continuación, se podrían incorporar técnicas más eficientes para el entrenamiento, como Negative Sampling, o utilizar corpus más amplios que permitan evaluar el modelo en escenarios más realistas.
